In [3]:
!pip install -q transformers ipyplot==1.1.0 ftfy
!pip install datasets -q
# !pip install peft -q
!pip install transformers -U -q
!pip install git+https://github.com/openai/CLIP.git
!pip install faiss-gpu
# !pip install accelerate -U -q

  Cloning https://github.com/openai/CLIP.git to /tmp/pip-req-build-f7bw68ih
  Running command git clone --filter=blob:none --quiet https://github.com/openai/CLIP.git /tmp/pip-req-build-f7bw68ih
  Resolved https://github.com/openai/CLIP.git to commit a1d071733d7111c9c014f024669f959182114e33
  Preparing metadata (setup.py) ... done
  Created wheel for clip: filename=clip-1.0-py3-none-any.whl size=1369497 sha256=5d38a7a2b295f47b70565a548d9f2cf259e91decf8c8a743c79648e3bbe08a22
  Stored in directory: /tmp/pip-ephem-wheel-cache-1hqpy89h/wheels/da/2b/4c/d6691fa9597aac8bb85d2ac13b112deb897d5b50f5ad9a37e4
Successfully built clip
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 19.0 MB/s eta 0:00:00:00:0100:01


In [4]:
from datasets import load_dataset
import clip
import torch
import math
import os
import torch
from torch.utils.data import Dataset, DataLoader
from tqdm import tqdm
import faiss
import numpy as np



In [5]:
import sys
PATH = '/kaggle/input/'
sys.path.insert(1, PATH)

In [6]:
from retrieval.createIndex import CreateModelAndIndex, getIndex, createIndex
from retrieval.getEmbeddings import customCollateFn, createEmbeddingsForIndex
from retrieval.imgAndCtgDataset import ImgAndCtgDataset
from retrieval.utils import (
    FindNearestImage,
    createModel,
    getDataset,
    searchImage2Image,
    searchImage2Text,
    searchText2Image,
)

# Load dataset from huggingface

In [7]:
dataset = getDataset()
dataset = dataset['train']
dataset

Generating train split:   0%|          | 0/45623 [00:00<?, ? examples/s]

Generating val split:   0%|          | 0/1158 [00:00<?, ? examples/s]

Dataset({
    features: ['image_id', 'image', 'width', 'height', 'objects'],
    num_rows: 45623
})

In [8]:
id2label = {0: 'shirt, blouse',
 1: 'top, t-shirt, sweatshirt',
 2: 'sweater',
 3: 'cardigan',
 4: 'jacket',
 5: 'vest',
 6: 'pants',
 7: 'shorts',
 8: 'skirt',
 9: 'coat',
 10: 'dress',
 11: 'jumpsuit',
 12: 'cape',
 13: 'glasses',
 14: 'hat',
 15: 'headband, head covering, hair accessory',
 16: 'tie',
 17: 'glove',
 18: 'watch',
 19: 'belt',
 20: 'leg warmer',
 21: 'tights, stockings',
 22: 'sock',
 23: 'shoe',
 24: 'bag, wallet',
 25: 'scarf',
 26: 'umbrella',
 27: 'hood',
 28: 'collar',
 29: 'lapel',
 30: 'epaulette',
 31: 'sleeve',
 32: 'pocket',
 33: 'neckline',
 34: 'buckle',
 35: 'zipper',
 36: 'applique',
 37: 'bead',
 38: 'bow',
 39: 'flower',
 40: 'fringe',
 41: 'ribbon',
 42: 'rivet',
 43: 'ruffle',
 44: 'sequin',
 45: 'tassel'}


In [9]:
customDataset = ImgAndCtgDataset(dataset)
customDataset[0]['categories']

[33, 10, 23]

In [10]:
device = "cuda" if torch.cuda.is_available() else "cpu"

In [11]:
model, preprocess = createModel(device)

100%|████████████████████████████████████████| 338M/338M [00:01<00:00, 188MiB/s]


In [12]:
OUTPUT_DIR = "./retrieval/embeddings"
BATCH_SIZE = 1024
if not os.path.exists(OUTPUT_DIR):
    os.makedirs(OUTPUT_DIR)

In [13]:
# createEmbeddingsForIndex(device)

In [14]:
def loadEmbeddings(device=None):
    if device is None:
        device = "cuda:2" if torch.cuda.is_available() else "cpu"
    model, _ = createModel(device=device)
    imageEmbeddings = []
    categories = []
    # если отсутствует хотя бы 1 сохраненный батч, то создаем батчи
    if not os.path.isfile("retrieval/embeddings/batch-0.pkl"):
        createEmbeddingsForIndex(device=device)
    all_files = os.listdir(OUTPUT_DIR)
    for embedding_file in tqdm(all_files):
        stored_embedding = torch.load(f"{OUTPUT_DIR}/{embedding_file}")
        imageEmbeddings.append(stored_embedding["embeddings"])
        categories.extend(stored_embedding["categories"])
    imageEmbeddings = torch.cat(imageEmbeddings).numpy()
#     categories = torch.cat(categories).numpy()
    faiss.normalize_L2(imageEmbeddings)

    category = clip.tokenize(["This is " + query for query in id2label.values()]).to(device)
    with torch.no_grad():
        textEmbeddings = model.encode_text(category.to(device)).detach().cpu().numpy()
    textEmbeddings = np.float32(textEmbeddings)
    faiss.normalize_L2(textEmbeddings)
    return imageEmbeddings, categories, textEmbeddings

In [15]:
imageEmbeddings, categories, textEmbeddings = loadEmbeddings(device)

100%|██████████| 45/45 [00:00<00:00, 139.29it/s]


In [16]:
textEmbeddings.shape

(46, 512)

In [17]:
imageEmbeddings.shape

(45623, 512)

In [18]:
len(categories)

45623

In [19]:
def create_index(embeddings, device=None):
    d = embeddings.shape[1]
    index = faiss.IndexFlatIP(d)
    assert index.is_trained
    index.add(embeddings)
    return index

In [45]:
class MetricsCalculator:
    
    def __init__(self, img_embeddings, categories, index, batch_size=1000):
        self.img_embeddings = img_embeddings
        self.index = index
        self.batch_size = batch_size
        self.categories = categories

   
    def compute_precision_at_k(self, predictions, true_categories, k):
        """ function to calculate metric precision at rank k for a query(a image)

        Args:
            predictions (list): list of predicted categories on the image
            true_categories (list): list of correct categories on the image
            k (int): rank for calculate

        Returns:
            precision@k
        """
        relevant_count = sum(1 for p in predictions if p in true_categories)
        precision = relevant_count / k
        return precision
    
    def compute_recall_at_k(self, predictions, true_categories, k):
        """function to calculate metric precision at rank k for a query(a image)

        Args:
            predictions (list): list of predicted categories on the image
            true_categories (list): list of correct categories on the image
            k (int): rank for calculate

        Returns:
            recall@k
        """
        relevant_count = sum(1 for p in predictions if p in true_categories)
        recall = relevant_count / len(true_categories)
        return recall
    
    def dcg_at_k(self, r, k):
        r = np.asfarray(r)[:k]
        if r.size:
            return r[0] + np.sum(r[1:] / np.log2(np.arange(2, r.size + 1)))
        return 0.

    def compute_ndcg_at_k(self, predictions, true_categories, k):
        """function to calculate metric precision at rank k for a query(a image)

        Args:
            predictions (list): list of predicted categories on the image
            true_categories (list): list of correct categories on the image
            k (int): rank for calculate

        Returns:
            ndcg@k
        """
        predictions = np.array(predictions)
        r = np.array([1 if cat in true_categories else 0 for cat in predictions])
        idcg = self.dcg_at_k(sorted(r, reverse=True), k)
        if not idcg:
            return 0.
        return self.dcg_at_k(r, k) / idcg

    def compute_metric(self, metric_name, metric_calc_func, top_k):
        """compute img2text rank metrics

        Args:
            metric_name (str): the name of the metric to calculate
            metric_calc_func (function): funtion to calculate metric
            top_k (int): _description_

        Returns:
            retrieval metric
        """
        _, indices = self.index.search(self.img_embeddings, top_k)
        metric = 0
        for i in tqdm(range(len(self.img_embeddings))):
            metric += metric_calc_func(indices[i], self.categories[i], top_k)
        print(f'{metric_name}@{top_k} = ', metric / len(self.img_embeddings))
        return metric / len(self.img_embeddings)
    
    def recall_calc(self, topk=[1, 5, 7, 10]):
        recall = []
        for k in topk:
            recall_at_k = self.compute_metric("recall", self.compute_recall_at_k, k)
            recall.append(recall_at_k)
        return recall
    
    def precision_calc(self, topk=[1, 5, 7, 10]):
        precision = []
        for k in topk:
            precision_at_k = self.compute_metric("precision", self.compute_precision_at_k, k)
            precision.append(precision_at_k)
        return precision
    
    def ndcg_calc(self, topk=[1, 5, 7, 10]):
        ndcg = []
        for k in topk:
            ndcg_at_k = self.compute_metric("ndcg", self.compute_ndcg_at_k, k)
            ndcg.append(ndcg_at_k)
        return ndcg
    def all_metrics_calc(self, topk=[1, 5, 7, 10]):
        return {"recall": self.recall_calc(topk),
                "precision": self.precision_calc(topk),
                "ndcg": self.ndcg_calc(topk)}

In [21]:
index = create_index(textEmbeddings, device)

In [46]:
metric = MetricsCalculator(imageEmbeddings, categories, index)

In [23]:
metric.compute_metric("recall", metric.compute_recall_at_k, 3)

100%|██████████| 45623/45623 [00:00<00:00, 207812.13it/s]

recall@3 =  0.3154861252999814


0.3154861252999814

In [24]:
metric.compute_metric("recall", metric.compute_recall_at_k, 5)

100%|██████████| 45623/45623 [00:00<00:00, 170427.70it/s]

recall@5 =  0.38405665015708723


0.38405665015708723

In [25]:
metric.compute_metric("recall", metric.compute_recall_at_k, 7)

100%|██████████| 45623/45623 [00:00<00:00, 140459.33it/s]

recall@7 =  0.43294589153697355


0.43294589153697355

In [38]:
metric.compute_metric("ndcg", metric.getNDCG, 3)

100%|██████████| 45623/45623 [00:02<00:00, 22309.95it/s]

ndcg@3 =  0.8303946492561459


0.8303946492561459

In [42]:
recall = metric.recall_calc()
recall

100%|██████████| 45623/45623 [00:00<00:00, 265577.37it/s]


recall@1 =  0.15817088618661695


100%|██████████| 45623/45623 [00:00<00:00, 168564.16it/s]


recall@5 =  0.38405665015708723


100%|██████████| 45623/45623 [00:00<00:00, 140824.94it/s]


recall@7 =  0.43294589153697355


100%|██████████| 45623/45623 [00:00<00:00, 111844.15it/s]

recall@10 =  0.4969707499659806


[0.15817088618661695,
 0.38405665015708723,
 0.43294589153697355,
 0.4969707499659806]

In [47]:
all_metrics = metric.all_metrics_calc()
all_metrics

100%|██████████| 45623/45623 [00:00<00:00, 267726.90it/s]


recall@1 =  0.15817088618661695


100%|██████████| 45623/45623 [00:00<00:00, 169690.33it/s]


recall@5 =  0.38405665015708723


100%|██████████| 45623/45623 [00:00<00:00, 140751.09it/s]


...

100%|██████████| 45623/45623 [00:02<00:00, 21162.43it/s]


ndcg@7 =  0.7869897158573829


100%|██████████| 45623/45623 [00:02<00:00, 19149.62it/s]

ndcg@10 =  0.7568686714063749


{'recall': [0.15817088618661695,
  0.38405665015708723,
  0.43294589153697355,
  0.4969707499659806],
 'precision': [0.6406637003265897,
  0.33926309098478613,
  0.2822385951946714,
  0.2347587839467248],
 'ndcg': [0.6406637003265897,
  0.8117664002809806,
  0.7869897158573829,
  0.7568686714063749]}